# Use this notebook to manipulate report files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import sys
DRIVE_DIR = "drive/My Drive/Springboard/capstone"
sys.path.append(DRIVE_DIR)


%tensorflow_version 2.x


import tensorflow as tf



import pandas as pd
import numpy as np
import importlib
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from util import dict_util as du
from util import file_util as fu
from util import plot_util as pu
from util import keras_util as ku
import util.report_util as ru


import pickle
import json
from datetime import datetime
import os

import matplotlib.pyplot as plt
import seaborn as sns
import logging
from pprint import pprint

pd.set_option('max_colwidth', 150)
logging.basicConfig(level=logging.INFO)

%matplotlib inline
sns.set()
# checl to make sure we are using GPU here




tf.test.gpu_device_name()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


''

# Read Report

In [0]:
report = pd.read_csv(ku.ModelWrapper.get_report_file_name(DRIVE_DIR), quotechar="'")
report

,model_name,architecture,description,accuracy,classification_report,confusion_matrix,embedding,evaluate_time_min,file,fpr,loss,max_sequence_length,model_file,model_json_file,network_history_file,predict_time_min,roc_auc,status,status_date,test_examples,test_features,tokenizer_file,tpr,train_examples,train_features,train_time_min,weights_file,sampling_type,epochs,batch_size,class_weight,feature_set_name
0,DNN,2400x2400,2 Layer DNN with 2/3 Feature Size Hidden Units Per Layer,0.685200,"{""1"": {""precision"": 0.6518439464010435, ""recall"": 0.7874230053001002, ""f1-score"": 0.7132476968989231, ""support"": 6981}, ""2"": {""precision"": 0.38979...","[[5497, 132, 509, 113, 730], [1570, 168, 698, 243, 546], [792, 106, 1280, 979, 1383], [232, 18, 622, 2163, 5263], [342, 7, 253, 1134, 25004]]",0,0.16,drive/My Drive/Springboard/capstone/data/review_body-tfidf-df_none-ngram13-199134-4000-nolda.csv,NaN,0.803525,4000,drive/My Drive/Springboard/capstone/models/review_body-tfidf-df_none-ngram13-199134-4000-nolda-DNN-4000-star_rating-model.h5,drive/My Drive/Springboard/capstone/models/review_body-tfidf-df_none-ngram13-199134-4000-nolda-DNN-4000-star_rating-model.json,NaN,0.14,"{""auc_1"": 0.9547194866365962, ""auc_2"": 0.8730325230419483, ""auc_3"": 0.8424719810975009, ""auc_4"": 0.7710448208420866, ""auc_5"": 0.8976579818946084, ...",success,2019-11-06 06:16:39,49784,4000,drive/My Drive/Springboard/capstone/models/dl-tokenizer.pkl,NaN,149350,4000,30.60,drive/My Drive/Springboard/capstone/models/review_body-tfidf-df_none-ngram13-199134-4000-nolda-DNN-4000-star_rating-weights.h5,none,57,128,NaN,tfidf_ngram13
1,DNNB,2400x2400,"2 Layer DNN with balaned weights, 2/3 Feature Size Hidden Units Per Layer",0.686787,"{""1"": {""precision"": 0.6273714223050774, ""recall"": 0.8195100988397078, ""f1-score"": 0.7106832298136646, ""support"": 6981}, ""2"": {""precision"": 0.36194...","[[5721, 122, 374, 95, 669], [1739, 156, 605, 198, 527], [947, 113, 1205, 831, 1444], [295, 32, 599, 1933, 5439], [417, 8, 225, 914, 25176]]",0,0.28,drive/My Drive/Springboard/capstone/data/review_body-tfidf-df_none-ngram13-199134-4000-nolda.csv,NaN,0.805356,4000,drive/My Drive/Springboard/capstone/models/DNN-2_.6x_.6x-tfidf-sampling_none-199134-4000-star_rating-model.h5,drive/My Drive/Springboard/capstone/models/DNN-2_.6x_.6x-tfidf-sampling_none-199134-4000-star_rating-model.json,NaN,0.24,"{""auc_1"": 0.9550678260034245, ""auc_2"": 0.8737652434328969, ""auc_3"": 0.8451384675048305, ""auc_4"": 0.7706660359869926, ""auc_5"": 0.897633918144561, ""...",success,2019-11-06 17:06:53,49784,4000,drive/My Drive/Springboard/capstone/models/dl-tokenizer.pkl,NaN,149350,4000,60.77,drive/My Drive/Springboard/capstone/models/DNN-2_.6x_.6x-tfidf-sampling_none-199134-4000-star_rating-weights.h5,none,73,128,"[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",tfidf_ngram13
2,DNN,6000x6000,2 Layer NN with the 1.5 features hidden units per layer,0.687128,"{""1"": {""precision"": 0.6211552888222055, ""recall"": 0.8302535453373442, ""f1-score"": 0.7106424717999018, ""support"": 6981}, ""2"": {""precision"": 0.36160...","[[5796, 120, 482, 88, 495], [1747, 162, 731, 198, 387], [974, 120, 1520, 848, 1078], [313, 35, 864, 2297, 4789], [501, 11, 443, 1352, 24433]]",0,0.12,drive/My Drive/Springboard/capstone/data/review_body-tfidf-df_none-ngram13-199134-4000-nolda.csv,NaN,0.803501,4000,drive/My Drive/Springboard/capstone/models/DNN-2_1.5x1-tfidf_ngram13-sampling_none-199134-4000-star_rating-model.h5,drive/My Drive/Springboard/capstone/models/DNN-2_1.5x1-tfidf_ngram13-sampling_none-199134-4000-star_rating-model.json,NaN,0.09,"{""auc_1"": 0.9550272882319519, ""auc_2"": 0.8727321789424136, ""auc_3"": 0.8448057390821067, ""auc_4"": 0.7705471154306127, ""auc_5"": 0.8973950016858256, ...",success,2019-11-07 20:12:30,49784,4000,drive/My Drive/Springboard/capstone/models/dl-tokenizer.pkl,NaN,149350,4000,54.59,drive/My Drive/Springboard/capstone/models/DNN-2_1.5x

In [0]:
report.columns

Index(['accuracy', 'architecture', 'classification_report', 'confusion_matrix',
       'description', 'embedding', 'evaluate_time_min', 'file', 'fpr', 'loss',
       'max_sequence_length', 'model_file', 'model_json_file', 'model_name',
       'network_history_file', 'predict_time_min', 'roc_auc', 'status',
       'status_date', 'test_examples', 'test_features', 'tokenizer_file',
       'tpr', 'train_examples', 'train_features', 'train_time_min',
       'weights_file', 'sampling_type', 'epochs', 'batch_size', 'class_weight',
       'feature_set_name', 'class_weights'],
      dtype='object')

In [0]:
report[["model_name", "architecture", "description", "class_weight" ]]

,model_name,architecture,description,class_weight
0,DNN,2_2400_2400,2 Layer DNN with 2/3 Feature Size Hidden Units Per Layer,NaN
1,DNNB,2_2400_2400,"2 Layer DNN with balaned weights, 2/3 Feature Size Hidden Units Per Layer","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]"
2,DNN,2_6000_6000,2 Layer NN with the 1.5 features hidden units per layer,NaN
3,CNNB,1_32,"1 Layer CNN with balanced weights, 32 filters, kernel size 3","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]"
4,LSTM,1_64,1 Layer 64 LSTM Units and 20% 1D Spatial Dropout,NaN
5,GRU,1_bidirectional,1 Layer Bi-directional GRU (250 units),NaN
6,GRU,1_bidirection_attention,1 Layer Bi-directional GRU (250 units) with attention,NaN
7,GRUB,1_bidirection_attention,"1 Layer Bi-directional GRU (250 units) with attention, balanced class weights","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]"


In [0]:
report.loc[[1, 2], "architecture"] = "2_2400_2400"
report.loc[7, "architecture"] = "2_6000_6000"
report.loc[7, "description"] = "2 Layer NN with the 1.5 features hidden units per layer"
report.loc[5, "model_name"] = "GRUB"


In [0]:
report.iloc[5]

accuracy                                                                                                                                                              0.686224
architecture                                                                                                                                           1_bidirection_attention
classification_report    {"1": {"precision": 0.6267744048918978, "recall": 0.8222317719524423, "f1-score": 0.7113204039903339, "support": 6981}, "2": {"precision": 0.30238...
confusion_matrix                  [[5740, 518, 351, 51, 321], [1660, 570, 580, 140, 275], [906, 518, 1336, 810, 970], [342, 183, 864, 1816, 5093], [510, 96, 475, 958, 24701]]
description                                                                                                              1 Layer Bi-directional GRU (250 units) with attention
embedding                                                                                                                    

# Save Report

In [0]:
report.to_csv(ku.ModelWrapper.get_report_file_name(DRIVE_DIR), index=False, quotechar="'")

# Looking at Report

In [26]:
import util.report_util as ru
import importlib
importlib.reload(ru)
report2 = ru.load_dnn_report(DRIVE_DIR)
report2[["model_name", "architecture", "description", "class_weight", "eval_metric" ]]

INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating met

,model_name,architecture,description,class_weight,eval_metric
0,DNN,2400x2400,2 Layer DNN with 2/3 Feature Size Hidden Units Per Layer,NaN,0.171430
1,DNNB,2400x2400,"2 Layer DNN with balaned weights, 2/3 Feature Size Hidden Units Per Layer","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",0.159874
2,DNN,6000x6000,2 Layer NN with the 1.5 features hidden units per layer,NaN,0.172488
3,CNN,1x32,"1 Layer CNN with 32 filters, kernel size 3",NaN,0.235893
4,CNNB,1x32,"1 Layer CNN with balanced weights, 32 filters, kernel size 3","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",0.254738
5,LSTM,1x64,1 Layer LSTM 64 Units and 20% 1D Spatial Dropout,NaN,0.304897
6,LSTM,1x500,1 Layer LSTM 500 Units and 20% 1D Spatial Dropout,NaN,0.310015
7,GRU-bi,1x250,1 Layer Bi-directional GRU (250 units),NaN,0.293265
8,GRU-bi,1x250_attention,1 Layer Bi-directional GRU (250 units) with attention,NaN,0.310342
9,GRUB-bi,1x250_attention,"1 Layer Bi-directional GRU (250 units) with attention, balanced class weights","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",0.240811


,model_name,architecture,description,class_weight,eval_metric
0,DNN,2400x2400,2 Layer DNN with 2/3 Feature Size Hidden Units Per Layer,NaN,0.171430
1,DNNB,2400x2400,"2 Layer DNN with balaned weights, 2/3 Feature Size Hidden Units Per Layer","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",0.159874
2,DNN,6000x6000,2 Layer NN with the 1.5 features hidden units per layer,NaN,0.172488
3,CNNB,1x32,"1 Layer CNN with balanced weights, 32 filters, kernel size 3","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",0.254738
4,LSTM,1x64,1 Layer 64 LSTM Units and 20% 1D Spatial Dropout,NaN,0.304897
5,GRU-bi,1x250,1 Layer Bi-directional GRU (250 units),NaN,0.293265
6,GRU-bi,1x250_attention,1 Layer Bi-directional GRU (250 units) with attention,NaN,0.310342
7,GRUB-bi,1x250_attention,"1 Layer Bi-directional GRU (250 units) with attention, balanced class weights","[1.4363387189844201, 3.078518976578805, 2.1884059563712293, 1.195282112845138, 0.37238709677419357]",0.240811


In [0]:
from pprint import pprint
pprint(json.loads(report.loc[0, "classification_report"]))

{'1': {'f1-score': 0.7082479508196722,
       'precision': 0.6404169079328315,
       'recall': 0.7921501217590603,
       'support': 6981},
 '2': {'f1-score': 0.1455637707948244,
       'precision': 0.28558476881233,
       'recall': 0.09767441860465116,
       'support': 3225},
 '3': {'f1-score': 0.3410925254415721,
       'precision': 0.36368171613868794,
       'recall': 0.3211453744493392,
       'support': 4540},
 '4': {'f1-score': 0.3410258376140458,
       'precision': 0.46870389884088515,
       'recall': 0.26801638949144374,
       'support': 8298},
 '5': {'f1-score': 0.8408464295561069,
       'precision': 0.7796880991946823,
       'recall': 0.912415856394914,
       'support': 26740},
 'accuracy': 0.6814438373774707,
 'macro avg': {'f1-score': 0.47535530284524424,
               'precision': 0.5076150781838834,
               'recall': 0.47828043213988164,
               'support': 49784},
 'weighted avg': {'f1-score': 0.6483277377840637,
                  'precision': 0.6

In [0]:
report = ru.load_dnn_report(DRIVE_DIR)

INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate
INFO:util.report_util:calculating metric from dictionary
INFO:util.report_util:got all values to calculate


In [0]:
report[["model_name", "architecture", "eval_metric", "epochs", "description"]]

,model_name,architecture,eval_metric,epochs,description
0,CNNB,1,0.254738,6,CNNB-1-embedding_random-batch_none-sampling_none-star_rating
1,DNN,2_.6x_.6x,0.171430,57,DNN-2_.6x_.6x-tfidf-batch_none-sampling_none-star_rating
2,DNNB,2_.6x_.6x,0.159874,73,DNN-2_.6x_.6x-tfidf-sampling_none-199134-4000-star_rating


In [0]:
report.columns

Index(['accuracy', 'architecture', 'classification_report', 'confusion_matrix',
       'description', 'embedding', 'evaluate_time_min', 'file', 'fpr', 'loss',
       'max_sequence_length', 'model_file', 'model_json_file', 'model_name',
       'network_history_file', 'predict_time_min', 'roc_auc', 'status',
       'status_date', 'test_examples', 'test_features', 'tokenizer_file',
       'tpr', 'train_examples', 'train_features', 'train_time_min',
       'weights_file', 'sampling_type', 'eval_metric'],
      dtype='object')

In [0]:
report.loc[2, "model_name"] = "DNNB"

In [0]:
report.loc[0, "epochs"] = 6
report.loc[1, "epochs"] = 57
report.loc[2, "epochs"] = 73

In [0]:
import numpy as np
report = report.astype({"epochs":np.int32})

In [0]:
report["batch_size"] = 128